In [ ]:
#imports
import numpy as np
import pandas as pd
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding,Flatten
from keras.layers import SimpleRNN
from keras import initializers
from sklearn.model_selection import train_test_split

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
# processing data set and filtering out columns that have no information
products = pd.read_csv("/content/drive/My Drive/Kaggle/combined/products.csv")
products = products.drop(["subhead"],1)

reviews = pd.read_csv("/content/drive/My Drive/Kaggle/combined/reviews.csv")
reviews=reviews.drop(["ingredients","texture", "likes","taste"],axis=1)



In [ ]:
print(products.shape)
print(reviews.shape)

(241, 7)
(21674, 9)


Preparing the data for training using train_test_split.
https://www.bitdegree.org/learn/train-test-split

In [ ]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
## split the data 
train, test = train_test_split(products, test_size=0.2, random_state=42)
print("Training data shape:", train.shape)
print("Test  shape:", test.shape)


Training data shape: (192, 7)
Test test shape: (49, 7)


In [ ]:
# variable for unique product key for ice cream brand and flavor
n_product = len(products.key.unique())

In [ ]:
max_features = 30000
maxlen = 50  
batch_size = 32

In [ ]:
# building RNN model
rnn_hidden_dim = 10
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
model_rnn.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

In [ ]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)